### Calculate Price of European and American Option
#### 1. BS model
#### 2. Monte Carlo
#### 3. Binomial Tree
#### 4. Binomial Tree with One Column Vector
#### 5. Binomial Tree with Combinatorics Method

In [1]:
# Calculate Price of European Put and Call Option by BS model
# Inputs: S0, K, r, q, σ, T
import numpy as np
import scipy.stats
import math

# Input Info
p = 50
k = 50
r = 0.1
q = 0.05
sd = 0.4
t = 0.5

# Black Scholes formula
call = 0
put = 0
d1 = (math.log(p/k)+(r-q+((sd**2)/2))*t)/(sd*(t**(1/2)))
print("d1 : ", d1)
d2 = (math.log(p/k)+(r-q-((sd**2)/2))*t)/(sd*(t**(1/2)))
print("d2 : ", d2)
print("Calculate Price of Option by BS model")
call = p*(np.exp(-q*t))*scipy.stats.norm(0, 1).cdf(d1) - k*(np.exp(-r*t))*scipy.stats.norm(0, 1).cdf(d2)
print("call price : ", round(call,4))

put = k*(np.exp(-r*t))*scipy.stats.norm(0, 1).cdf(-d2) - p*(np.exp(-q*t))*scipy.stats.norm(0, 1).cdf(-d1)
print("put price : ", round(put,4))



d1 :  0.2298097038856279
d2 :  -0.05303300858899108
Calculate Price of Option by BS model
call price :  6.0396
put price :  4.8356


In [2]:
# Calculate Price of European Option by Monte Carlo
from scipy.stats import norm
import math
import numpy as np
size = 100
# Input Info
p1 = 50
k1 = 50
r1 = 0.1
q1 = 0.05
sd1 = 0.4
t1 = 0.5

# call 
return_l = []
ans_l = []
mean = math.log(p1)+(r1-q1-((sd1**2)/2))*t1
var = (sd1**2)*t1
# print("mean",mean)
# print("var",var)

for i in range(20):
    rd_sample_l = []
    return_l = []
    for count_t in range(1):
        r = np.random.standard_normal(size) #scale是標準差
        for j in range(size):
            rd_sample_l.append(np.exp(mean+float(r[j])*(var**0.5)))
        for h in range(size):
            if rd_sample_l[h] >= k1:
                return_l.append(rd_sample_l[h] - k1)
            else:
                return_l.append(0)
    #print(len(return_l))
    ans_l.append(np.mean(return_l)*np.exp(-r1*t1))
ans_mean = np.mean(ans_l)
ans_std = np.std(ans_l)

# call 
return_l = []
ans_l = []
return_l2 = []
ans_l2 = []
mean = math.log(p1)+(r1-q1-((sd1**2)/2))*t1
var = (sd1**2)*t1
# print("mean",mean)
# print("var",var)
size = 10000
for i in range(20):
    rd_sample_l = []
    return_l = []
    return_l2 = []
    for count_t in range(1):
        r = np.random.standard_normal(10000) #scale是標準差
        for j in range(size):
            rd_sample_l.append(np.exp(mean+float(r[j])*(var**0.5)))
        for h in range(size):
            return_l.append(max(rd_sample_l[h] - k1,0))
            return_l2.append(max(k1 - rd_sample_l[h] ,0))
    #print(len(return_l))
    ans_l.append(np.mean(return_l)*np.exp(-r1*t1))
    ans_l2.append(np.mean(return_l2)*np.exp(-r1*t1))
ans_mean = np.mean(ans_l)
ans_std = np.std(ans_l)
ans_mean2 = np.mean(ans_l2)
ans_std2 = np.std(ans_l2)
print("Calculate Price of European Option by Monte Carlo")
print("call price : ",round(ans_mean,4))
print("95% interval of call price : ",ans_mean- 2*ans_std,ans_mean+2*ans_std)
print("put price : ",round(ans_mean2,4))
print("95% interval of put price : ",ans_mean2- 2*ans_std2,ans_mean2+2*ans_std2)

Calculate Price of European Option by Monte Carlo
call price :  6.0639
95% interval of call price :  5.876549884205755 6.251276520610009
put price :  4.8317
95% interval of put price :  4.725522037815822 4.937820791995356


In [3]:
# Calculate Price of European and American Option by Binomial Tree
import numpy as np

# Input Info
option_type = 'p'
option_type1 = 'e'
s_p = 50
q = 0.05
sd1 = 0.4
r = 0.1
strike = 50
t = 0.5
period = 100

delt = t / period
u = np.exp(sd1 * (delt**0.5))   
d = 1/u         
# pre-calculate
p = (np.exp((r-q)*delt) - d)/(u - d)

s_price = [] # list for possible stock price
o_price = [] # list for possible option price
probability_rate = []
for i in range(period+1):
    s_price.append([])
    o_price.append([])
    probability_rate.append([])
# stock prices and probabilities for all periods
for j in range(period+1):
    for g in range(j+1):
        s_price[j].append(s_p*(u**(j-g))*((d)**(g))) # Calculate Final Stock Price
        
# final return for call option                 
if option_type == "c":
    for i in range(len(s_price[-1])):
        if s_price[-1][i]-strike > 0:
            return_of_option = s_price[-1][i]-strike
        else:
            return_of_option = 0
        o_price[-1].append(return_of_option) # Calculate Final Payoff of Option

# final return for put option
if option_type == "p":
    for i in range(len(s_price[-1])):
        if strike - s_price[-1][i] > 0:
            return_of_option = strike - s_price[-1][i]
        else:
            return_of_option = 0
        o_price[-1].append(return_of_option)  # Calculate Final Payoff of Option
        
# euro option
if option_type1 == 'e':
    for i in range(period):
        for j in range(period-i):
            o_price[period-i-1].append((o_price[period-i][j]*p + o_price[period-i][j+1]*(1-p))*np.exp(-r*delt)) 
# american option
if option_type1 == 'a':
    if option_type == "c":
        for i in range(period):
            for j in range(period-i):
                o_price[period-i-1].append(max([float(s_price[period-i-1][j]-strike),(o_price[period-i][j]*p + o_price[period-i][j+1]*(1-p))*np.exp(-r*delt)]))
    if option_type == "p":
        for i in range(period):
            for j in range(period-i):
                o_price[period-i-1].append(max([float(strike - s_price[period-i-1][j]),(o_price[period-i][j]*p + o_price[period-i][j+1]*(1-p))*np.exp(-r*delt)]))
# print("o_price ",o_price[-1])
# print("s_price ",s_price[-1])
print("Calculate Price of Option by Binomial Tree")
print("call or put : ", option_type)
print("euro or amer : ", option_type1)
print("option value is ",round(o_price[0][0],4))

Calculate Price of Option by Binomial Tree
call or put :  p
euro or amer :  e
option value is  4.822


In [4]:
# Calculate Price of European and American Option by Binomial Tree with One Column Vector.
import numpy as np

# Input Info
option_type = 'p'
option_type1 = 'a'
s_p = 50
q = 0.05
sd1 = 0.4
r = 0.1
strike = 50
t = 0.5
period = 500

delt = t / period
u = np.exp(sd1 * (delt**0.5))   
d = 1/u         
# pre-calculate
p = (np.exp((r-q)*delt) - d)/(u - d)

s_price = [] # list for possible stock price
o_price = [] # list for possible option price
probability_rate = []

# stock prices and probabilities for all periods
for j in range(period+1):
    s_price.append(s_p*(u**(period-j))*((d)**(j)))
        
# final return for call option                 
if option_type == "c":
    for i in range(len(s_price)):
        if s_price[i]-strike > 0:
            return_of_option = s_price[i]-strike
        else:
            return_of_option = 0
        o_price.append(return_of_option) # Calculate Final Payoff of Option

# final return for put option
if option_type == "p":
    for i in range(len(s_price)):
        if strike - s_price[i] > 0:
            return_of_option = strike - s_price[i]
        else:
            return_of_option = 0
        o_price.append(return_of_option) # Calculate Final Payoff of Option
        
# euro option
if option_type1 == 'e':
    for j in range(period):
        for i in range(period):
            o_price[i] = (o_price[i]*p + o_price[i+1]*(1-p))*np.exp(-r*delt) # One Column Vector for Option
# american option
if option_type1 == 'a':
    if option_type == "c":
        for j in range(period):
            for i in range(period):
                o_price[i] = max([float(s_price[i]*u**(-j-1)-strike),(o_price[i]*p + o_price[i+1]*(1-p))*np.exp(-r*delt)]) # One Column Vector
    if option_type == "p":
        for j in range(period):
            for i in range(period):
                o_price[i] = max([float(strike - s_price[i]*u**(-j-1)),(o_price[i]*p + o_price[i+1]*(1-p))*np.exp(-r*delt)]) # One Column Vector
# print("o_price ",o_price)
# print("s_price ",s_price)
print("Calculate Price of Option by Binomial Tree with One Column Vector")
print("call or put : ", option_type)
print("euro or amer : ", option_type1)
print("option value is ",round(o_price[0],4))

Calculate Price of Option by Binomial Tree with One Column Vector
call or put :  p
euro or amer :  a
option value is  4.9859


In [6]:
# Calculate Price of European and American Option by Binomial Tree with Combinatorics.
import numpy as np

# Input Info
option_type = 'c'
option_type1 = 'e'
s_p = 50
q = 0.05
sd1 = 0.4
r = 0.1
strike = 50
t = 0.5
period = 500

delt = t / period
u = np.exp(sd1 * (delt**0.5))   
d = 1/u         
# pre-calculate
p = (np.exp((r-q)*delt) - d)/(u - d)

s_price = [] # list for possible stock price
o_price = [] # list for possible option price
probability_rate = []

# stock prices and probabilities for all periods
for j in range(period+1):
    s_price.append(s_p*(u**(period-j))*((d)**(j))) # Calculate Final Stock Price with Combinatorics
        
# final return for call option                 
if option_type == "c":
    for i in range(len(s_price)):
        if s_price[i]-strike > 0:
            return_of_option = s_price[i]-strike
        else:
            return_of_option = 0
        o_price.append(return_of_option) # Calculate Final Payoff of Option

# final return for put option
if option_type == "p":
    for i in range(len(s_price)):
        if strike - s_price[i] > 0:
            return_of_option = strike - s_price[i]
        else:
            return_of_option = 0
        o_price.append(return_of_option) # Calculate Final Payoff of Option
        
# euro option
if option_type1 == 'e':
    for j in range(period):
        for i in range(period):
            o_price[i] = (o_price[i]*p + o_price[i+1]*(1-p))*np.exp(-r*delt) # One Column Vector for Option
# american option
if option_type1 == 'a':
    if option_type == "c":
        for j in range(period):
            for i in range(period):
                o_price[i] = max([float(s_price[i]*u**(-j-1)-strike),(o_price[i]*p + o_price[i+1]*(1-p))*np.exp(-r*delt)]) # One Column Vector
    if option_type == "p":
        for j in range(period):
            for i in range(period):
                o_price[i] = max([float(strike - s_price[i]*u**(-j-1)),(o_price[i]*p + o_price[i+1]*(1-p))*np.exp(-r*delt)]) # One Column Vector
# print("o_price ",o_price)
# print("s_price ",s_price)
print("Calculate Price of Option by Binomial Tree with One Column Vector")
print("call or put : ", option_type)
print("euro or amer : ", option_type1)
print("option value is ",round(o_price[0],4))

Calculate Price of Option by Binomial Tree with One Column Vector
call or put :  c
euro or amer :  e
option value is  6.0369


In [7]:
# Calculate Price of European and American Option by Binomial Tree with Combinatorics.
import numpy as np
from decimal import Decimal #精于计算
from decimal import getcontext #保留的小数位数，自己设置
# Input Info
getcontext().prec = 100000
s_p = 50
q = 0.05
sd1 = 0.4
r = 0.1
strike = 50
t = 0.5
period = 2000

delt = t / period
u = np.exp(sd1 * (delt**0.5))   
d = 1/u         
# pre-calculate
p = (np.exp((r-q)*delt) - d)/(u - d)

s_price = [] # list for possible stock price
o_price = [] # list for possible option price
o_price1 = [] # list for possible option price
pro_l2 = []

pro_l = []
p_l = []
for i in range(period):
    p_l.append(Decimal(p**(period-i))*Decimal((1-p)**(i))) # Calculate Final Payoff of Option
    
# stock prices and probabilities for all periods
for j in range(period):
    s_price.append(s_p*(u**(period-j))*((d)**(j))) # Calculate Final Stock Price with Combinatorics
for j in range(int(period/2+1)):
    pro_l2.append((np.math.factorial(period))//(np.math.factorial(period-j)*np.math.factorial(j)))
for j in range(int(period/2+1)):
    pro_l.append(pro_l2[j]) 
for j in range(int(period/2)):
    pro_l.append(pro_l2[-j-2]) 
    
# final return for call option                 
for i in range(len(s_price)):
    o_price.append(max(Decimal(s_price[i]-strike)*p_l[i],0)*Decimal(pro_l[i])) # Calculate Final Payoff of Option

# final return for put option
for i in range(len(s_price)):
    o_price1.append(max(Decimal(strike-s_price[i])*p_l[i],0)*Decimal(pro_l[i])) # Calculate Final Payoff of Option
call = np.sum(o_price)
put = np.sum(o_price1)
print("Calculate Price of Option by Binomial Tree with One Column Vector")
print("option value is ",round(call,10))
print("option value is ",round(put,10))

Calculate Price of Option by Binomial Tree with One Column Vector
option value is  6.3143657704
option value is  5.0713419453
